In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import warnings

In [2]:
warnings.filterwarnings("ignore")

In [3]:
dataset=pd.read_csv("insurance_pre.csv")

In [4]:
dataset=pd.get_dummies(dataset,dtype=int,drop_first=True)

In [5]:
dataset.columns

Index(['age', 'bmi', 'children', 'charges', 'sex_male', 'smoker_yes'], dtype='object')

In [6]:
indep=dataset[['age','bmi','children','sex_male', 'smoker_yes']]

In [7]:
dep=dataset[['charges']]

In [8]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(indep,dep,test_size=0.30,random_state=0)

In [9]:
from sklearn.preprocessing import StandardScaler
#StandardScaler for X_train
sc=StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.transform(X_test)


In [10]:
#StandardScaler for y_train
scy=StandardScaler()
y_train=scy.fit_transform(y_train)
y_test=scy.transform(y_test)

In [13]:
filename1="sc.pkl"
filename2="scy.pkl"
pickle.dump(sc,open(filename1,'wb'))
pickle.dump(scy,open(filename2,'wb'))

In [14]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
param_grid = {'criterion':['absolute_error', 'poisson', 'friedman_mse', 'squared_error'],
              'max_features': ['sqrt','log2'],
              'splitter':['best','random']}
grid = GridSearchCV(DecisionTreeRegressor(), param_grid, refit = True, verbose = 3,n_jobs=-1)
grid.fit(indep,dep)

Fitting 5 folds for each of 16 candidates, totalling 80 fits


GridSearchCV(estimator=DecisionTreeRegressor(), n_jobs=-1,
             param_grid={'criterion': ['absolute_error', 'poisson',
                                       'friedman_mse', 'squared_error'],
                         'max_features': ['sqrt', 'log2'],
                         'splitter': ['best', 'random']},
             verbose=3)

In [15]:
re=grid.cv_results_
grid_predictions=grid.predict(indep)

In [16]:
from sklearn.metrics import r2_score
r_score=r2_score(dep,grid_predictions)
print("The R_score value of best parameter{}".format(grid.best_params_),r_score)

The R_score value of best parameter{'criterion': 'poisson', 'max_features': 'log2', 'splitter': 'best'} 0.998667156135576


In [17]:
filename="finalized_model_Decision_tree.sav"

pickle.dump(grid,open(filename,'wb'))


In [18]:
age_input=float(input("Age:"))
bmi_input=float(input("BMI:"))
children_input=float(input("Children:"))
sex_male_input=int(input("Sex Male 0 or 1:"))
smoker_yes_input=int(input("Smoker Yes 0 or 1:"))

Age: 28
BMI: 33.75
Children: 2
Sex Male 0 or 1: 1
Smoker Yes 0 or 1: 1


In [19]:
preinput=sc.transform([[age_input,bmi_input,children_input,sex_male_input,smoker_yes_input]])

In [20]:
preinput

array([[-0.81907079,  0.48098694,  0.76341038,  1.02378711,  1.98149332]])

In [21]:
loaded_model=pickle.load(open("finalized_model_Decision_tree.sav",'rb'))
result=loaded_model.predict(preinput)

In [22]:
result

array([12829.4551])

In [23]:
preoutput=scy.inverse_transform([result])

In [24]:
preoutput

array([[1.52347134e+08]])